# Malware Image Classification 

Simple Classifier를 통해 Malware Image를 분류해보자.

* 이전에 작성했던 코드(github)를 참고해도 좋고
* 선대 연구자들이 연구한 자료 또는 코드를 참고해도 좋다.
* 뭐가 되도 좋으니 하나만 한 번 제대로 해보자!

## # 1. Import Library & Load Data

In [1]:
import os
import pickle
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [22]:
with open('../data.pkl', 'rb') as f:
    data = pickle.load(f)  # pickle.load()
    

불러온 데이터를 확인해본다.

In [23]:
data

(array([[ 94.,  32.,   2., ..., 129., 102., 124.],
        [ 54.,  29.,   6., ..., 106., 119., 100.],
        [ 54.,  29.,   6., ..., 120., 126., 104.],
        ...,
        [ 65.,  22.,  59., ..., 117., 116.,  98.],
        [ 65.,  22.,  59., ..., 117., 116.,  98.],
        [ 65.,  22.,  59., ..., 117., 116.,  98.]]),
 array([ 0.,  0.,  0., ..., 24., 24., 24.]),
 ['Adialer.C',
  'Agent.FYI',
  'Allaple.A',
  'Allaple.L',
  'Alueron.gen!J',
  'Autorun.K',
  'C2LOP.gen!g',
  'C2LOP.P',
  'Dialplatform.B',
  'Dontovo.A',
  'Fakerean',
  'Instantaccess',
  'Lolyda.AA1',
  'Lolyda.AA2',
  'Lolyda.AA3',
  'Lolyda.AT',
  'Malex.gen!J',
  'Obfuscator.AD',
  'Rbot!gen',
  'Skintrim.N',
  'Swizzor.gen!E',
  'Swizzor.gen!I',
  'VB.AT',
  'Wintrim.BX',
  'Yuner.A'],
 [122,
  116,
  2949,
  1591,
  198,
  106,
  200,
  146,
  177,
  162,
  381,
  431,
  213,
  184,
  123,
  159,
  136,
  142,
  158,
  80,
  128,
  132,
  408,
  97,
  800])

In [25]:
data[0]

array([[ 94.,  32.,   2., ..., 129., 102., 124.],
       [ 54.,  29.,   6., ..., 106., 119., 100.],
       [ 54.,  29.,   6., ..., 120., 126., 104.],
       ...,
       [ 65.,  22.,  59., ..., 117., 116.,  98.],
       [ 65.,  22.,  59., ..., 117., 116.,  98.],
       [ 65.,  22.,  59., ..., 117., 116.,  98.]])

In [26]:
malimg[1]

array([ 54.,  29.,   6., ..., 106., 119., 100.])

In [43]:
malimg[1]

array([ 0.,  0.,  0., ..., 24., 24., 24.])

In [44]:
malimg[2]

['Adialer.C',
 'Agent.FYI',
 'Allaple.A',
 'Allaple.L',
 'Alueron.gen!J',
 'Autorun.K',
 'C2LOP.gen!g',
 'C2LOP.P',
 'Dialplatform.B',
 'Dontovo.A',
 'Fakerean',
 'Instantaccess',
 'Lolyda.AA1',
 'Lolyda.AA2',
 'Lolyda.AA3',
 'Lolyda.AT',
 'Malex.gen!J',
 'Obfuscator.AD',
 'Rbot!gen',
 'Skintrim.N',
 'Swizzor.gen!E',
 'Swizzor.gen!I',
 'VB.AT',
 'Wintrim.BX',
 'Yuner.A']

In [45]:
malimg[3]

[122,
 116,
 2949,
 1591,
 198,
 106,
 200,
 146,
 177,
 162,
 381,
 431,
 213,
 184,
 123,
 159,
 136,
 142,
 158,
 80,
 128,
 132,
 408,
 97,
 800]

### Import Torch Library

In [9]:
# 참고(Reference)
# 나의 깃헙
# https://github.com/Steve-YJ/deep-learning-from-scratch-studying/blob/master/02_PyTorch_Introduction_Basic/Ch05_5.5_%EC%99%80%EC%9D%B8%20%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb


# PyTorch 라이브러리 임포트
import torch
from torch.autograd import Variable  # Variable?
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# scikit-learn 라이브러리 임포트
# from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split  # sklearn.model_selection

# Pandas 라이브러리 임포트
import pandas as pd

### Load Data

In [27]:
# Malimg 데이터 읽어들이기

malimg = data
# malimg

(array([[ 94.,  32.,   2., ..., 129., 102., 124.],
        [ 54.,  29.,   6., ..., 106., 119., 100.],
        [ 54.,  29.,   6., ..., 120., 126., 104.],
        ...,
        [ 65.,  22.,  59., ..., 117., 116.,  98.],
        [ 65.,  22.,  59., ..., 117., 116.,  98.],
        [ 65.,  22.,  59., ..., 117., 116.,  98.]]),
 array([ 0.,  0.,  0., ..., 24., 24., 24.]),
 ['Adialer.C',
  'Agent.FYI',
  'Allaple.A',
  'Allaple.L',
  'Alueron.gen!J',
  'Autorun.K',
  'C2LOP.gen!g',
  'C2LOP.P',
  'Dialplatform.B',
  'Dontovo.A',
  'Fakerean',
  'Instantaccess',
  'Lolyda.AA1',
  'Lolyda.AA2',
  'Lolyda.AA3',
  'Lolyda.AT',
  'Malex.gen!J',
  'Obfuscator.AD',
  'Rbot!gen',
  'Skintrim.N',
  'Swizzor.gen!E',
  'Swizzor.gen!I',
  'VB.AT',
  'Wintrim.BX',
  'Yuner.A'],
 [122,
  116,
  2949,
  1591,
  198,
  106,
  200,
  146,
  177,
  162,
  381,
  431,
  213,
  184,
  123,
  159,
  136,
  142,
  158,
  80,
  128,
  132,
  408,
  97,
  800])

malware sample에 대한 vector값

In [28]:
data = malimg[0]
data

array([[ 94.,  32.,   2., ..., 129., 102., 124.],
       [ 54.,  29.,   6., ..., 106., 119., 100.],
       [ 54.,  29.,   6., ..., 120., 126., 104.],
       ...,
       [ 65.,  22.,  59., ..., 117., 116.,  98.],
       [ 65.,  22.,  59., ..., 117., 116.,  98.],
       [ 65.,  22.,  59., ..., 117., 116.,  98.]])

In [29]:
data.shape

(9339, 50176)

* 정리
    * 9339개의 Malware Image Sample 데이터
    * 각각의 데이터 샘플은 50176 vector로 이루어져있다. (224x224)


## #2. Data Preprocessing

* NumPy to DataFrame 
* Data Preprocessing

### NumPy to DataFrame
malimg data타입은 NumPy배열이므로 한 번에 내용을 파악하기 어렵다.
pandas library를 이용해 데이터를 파악해보자.

In [30]:
df = pd.DataFrame(data)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,50166,50167,50168,50169,50170,50171,50172,50173,50174,50175
0,94.0,32.0,2.0,3.0,0.0,98.0,40.0,30.0,4.0,6.0,...,111.0,170.0,155.0,138.0,155.0,150.0,75.0,129.0,102.0,124.0
1,54.0,29.0,6.0,35.0,49.0,147.0,53.0,50.0,1.0,43.0,...,102.0,78.0,132.0,115.0,147.0,96.0,112.0,106.0,119.0,100.0
2,54.0,29.0,6.0,35.0,49.0,147.0,53.0,50.0,1.0,43.0,...,145.0,103.0,85.0,126.0,112.0,120.0,108.0,120.0,126.0,104.0
3,94.0,32.0,2.0,3.0,0.0,98.0,40.0,30.0,4.0,6.0,...,91.0,186.0,160.0,118.0,161.0,170.0,129.0,100.0,92.0,133.0
4,95.0,32.0,2.0,3.0,0.0,98.0,40.0,31.0,4.0,6.0,...,79.0,94.0,173.0,113.0,113.0,107.0,116.0,101.0,126.0,129.0


feature의 개수가 몇 개 안될때는 확인이 가능하지만 이미지 데이터처럼 feature의 개수가 50172개일 경우에는 효과적인지 모르겠다.

In [31]:
# 목적변수 데이터 출력
target = malimg[1]
target

array([ 0.,  0.,  0., ..., 24., 24., 24.])

### Data Preprocessing
* 설명변수 목적변수 분할
* Train_Test_Split

Q. 이미 데이터 전처리를 통해 설명변수와 목적변수를 분할해 두었다.

In [32]:
data

array([[ 94.,  32.,   2., ..., 129., 102., 124.],
       [ 54.,  29.,   6., ..., 106., 119., 100.],
       [ 54.,  29.,   6., ..., 120., 126., 104.],
       ...,
       [ 65.,  22.,  59., ..., 117., 116.,  98.],
       [ 65.,  22.,  59., ..., 117., 116.,  98.],
       [ 65.,  22.,  59., ..., 117., 116.,  98.]])

In [60]:
target

array([ 54.,  29.,   6., ..., 106., 119., 100.])

### Train_Test Split

데이터 집합을 훈련 데이터와 테스트 데이터로 분할한다.

In [33]:
# 데이터 집합을 훈련 데이터와 테스트 데이터로 분할
train_X, test_X, train_Y, test_Y = train_test_split(data, target, test_size = 0.2)

print(len(train_X))  # train data에서 설명변수X
print(len(test_X))   # test data에서 설명변수X

7471
1868


Train_Test Split 정리<br>
train 데이터의 설명변수의 변수명은 train_X, 목적변수의 변수명은 train_Y
test 데이터에서는?<br>
test 데이터의 설명변수의 변수명은 test_X, 목적변수명은 test_Y로 한다.

## #3. Make Tensor(텐서 생성)

완성된 train, test 데이터를 PyTorch의 tensor 데이터로 변환해준다.<br>
즉 모델의 학습 준비가 끝난 데이터를 파이토치가 다룰 수 있는 형태로 정리한다.
* 훈련 데이터, 테스트 데이터 tensor 변환
* train데이터의 X와 Y데이터 tensorDataset으로 병합
* Mini-Batch


### 훈련 데이터, 테스트 데이터 tensor 변환
<code>torch.from_numpy()</code>

In [34]:
# 훈련 데이터 텐서 변환
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

# 테스트 데이터 텐서 변환
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

# 텐서로 변환한 데이터 건수 확인
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

torch.Size([7471, 50176])
torch.Size([7471])
torch.Size([1868, 50176])
torch.Size([1868])


여기까지 Train_Test Split --20.04.16.Thur.pm 4:14--